#### [LangChain Handbook](https://pinecone.io/learn/langchain)

# LangChain Expression Language (LCEL)

The **L**ang**C**hain **E**xpression **L**anguage (LCEL) is a abstraction of some interesting Python concepts into a format that enables a "minimalist" code layer for building chains of LangChain components.

LCEL comes with strong support for:

1. Superfast development of chains.
2. Advanced features such as streaming, async, parallel execution, and more.
3. Easy integration with LangSmith and LangServe.

In [ ]:
#!pip uninstall -y langchain langchain-core langchain-community langchain-cohere

In [ ]:
# !pip install langchain-core langchain langchain-community langchain-cohere

In [ ]:
!pip install -qU \
    langchain==0.3.1 \
    langchain-openai \
    anthropic==0.7.7 \
    cohere==4.37 \
    docarray==0.39.1\
    langchain_chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 808.1/808.1 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.4/265.4 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.1 MB/s eta 0:00:

## LCEL Syntax

To understand LCEL syntax let's first build a simple chain in typical Python syntax.

In [ ]:
# from langchain.chat_models import ChatAnthropic
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_openai import ChatOpenAI
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.environ["OPENAI_API_KEY"] = open('/content/drive/MyDrive/Generative AI Content/key.txt','r').read()

In [ ]:
# Creating an instance of ChatOpenAI with the model set to 'gpt-4-turbo'
llm = ChatOpenAI(model='gpt-4-turbo')

In [ ]:
# ANTHROPIC_API_KEY = "<<YOUR_ANTHROPIC_API_KEY>>"

# if ANTHROPIC_API_KEY == "<<YOUR_ANTHROPIC_API_KEY>>":
#     raise ValueError("Please set your ANTHROPIC_API_KEY")


prompt = ChatPromptTemplate.from_template(
    "Give me small report about {topic}"
)

# model = ChatAnthropic(
#     model="claude-2.1",
#     max_tokens_to_sample=512,
#     anthropic_api_key=ANTHROPIC_API_KEY
# )  # swap Anthropic for OpenAI with `ChatOpenAI` and `openai_api_key`

output_parser = StrOutputParser()

In [ ]:
lcel_chain = prompt | llm | output_parser

# and run
out = lcel_chain.invoke({"topic": "Artificial Intelligence"})
print(out)

### Artificial Intelligence: An Overview

**Introduction**  
Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think like humans and mimic their actions. This field intersects with computer science and involves the creation and application of algorithms in a dynamic computing environment to perform tasks commonly associated with intelligent beings.

**Historical Context**  
The concept of AI originated with the ancient Greeks, but the foundation of the modern AI we recognize today started in the mid-20th century. The formal founding of AI as an academic discipline is dated to 1956, during a conference at Dartmouth College, led by John McCarthy who is known as the father of AI.

**Key Technologies**  
1. **Machine Learning (ML)**: This involves algorithms that allow software to become more accurate in predicting outcomes without being explicitly programmed. It uses statistical methods to enable machines to improve with experie

Pretty cool, the way that this pipe operator is being used is that it is taking output from the function to the _left_ and feeding it into the function on the _right_.

## How the Pipe Operator Works

To really understand LCEL we can take a look at how this pipe operation works. We know it takes output from the _right_ and feeds it to the _left_ — but this isn't typical Python, so how is this implemented? Let's try creating our own version of this with some simple functions.

We will be using the `__or__` method within Python class objects. When we place two classes together like `chain = class_a | class_b` the Python interpreter will check if these classes contain this `__or__` method. If they do then our `|` code will be translated into `chain = class_a.__or__(class_b)`.

That means both of these patterns will return the same thing:

```python
# object approach
chain = class_a.__or__(class_b)
chain("some input")

# pipe approach
chain = class_a | class_b
chain("some input")

```

With that in mind, we can build a `Runnable` class that consumes a function and turns it into a function that can be chained with other functions using the pipe operator `|`.

In [ ]:
class Runnable:
    def __init__(self, func):
        self.func = func

    def __or__(self, other):
        def chained_func(*args, **kwargs):
            # the other func consumes the result of this func
            return other(self.func(*args, **kwargs))
        return Runnable(chained_func)

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)

Let's implement this to take the value `3`, add `5` (giving `8`), and multiply by `2` (giving `16`).

> Add blockquote



In [ ]:
def add_five(x):
    return x + 5

def multiply_by_two(x):
    return x * 2

# wrap the functions with Runnable
add_five = Runnable(add_five)
multiply_by_two = Runnable(multiply_by_two)

# run them using the object approach
chain = add_five.__or__(multiply_by_two)
chain(3)  # should return 16

16

Using `__or__` directly we get the correct answer, now let's try using the pipe operator `|` to chain them together:

In [ ]:
# chain the runnable functions together
chain = add_five | multiply_by_two

# invoke the chain
chain(3)  # we should return 16

16

Using either method we get the same response, at it's core this is what LCEL is doing, but there is more.

## LCEL Deep Dive

Now that we understand what this syntax is doing under the hood, let's explore it within the context of LCEL and see a few of the additional methods that LangChain has provided to maximize flexibility when working with LCEL.

### Runnables

When working with LCEL we may find that we need to modify the structure or values being passed between components — for this we can use _runnables_. Let's try:

In [ ]:
! pip install langchain_community
! pip install -U langchain-cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.1
    Uninstalling langchain-0.3.1:
      Successfully uninstalled langchain-0.3.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.9/252.9 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: cohere
    Found existing installation: cohere 4.37
    Uninstalling cohere-4.37:
      Successfully uninstalled cohere-4.37


In [ ]:
# from langchain.embeddings import CohereEmbeddings
# from langchain.vectorstores import DocArrayInMemorySearch
import cohere
from langchain_chroma import Chroma


api_key = '1XyHUHdLz726y08PVVSg3YQ7UD3BsRq8SxYpPjG9'


# embedding = CohereEmbeddings(
#     model="embed-english-light-v3.0",
#     cohere_api_key=api_key
# )

import langchain_cohere
from langchain_cohere import CohereEmbeddings

embedding = CohereEmbeddings(
    model="embed-english-light-v3.0",
    cohere_api_key=api_key
)

# vecstore_a = DocArrayInMemorySearch.from_texts(
#     ["half the info will be here", "James' birthday is the 7th December"],
#     embedding=embedding,

# )
# vecstore_b = DocArrayInMemorySearch.from_texts(
#     ["and half here", "James was born in 1994"],
#     embedding=embedding
# )

In [ ]:
from langchain.docstore.document import Document

docs_a = [Document(page_content=doc) for doc in ["half the info will be here", "James' birthday is the 7th December"]]
docs_b = [Document(page_content=doc) for doc in ["and half here", "James was born in 1994"]]

In [ ]:
chroma_db_a = Chroma.from_documents(documents=docs_a,
                                  embedding=embedding)
chroma_db_b = Chroma.from_documents(documents=docs_b, collection_name='wiki_db',
                                  embedding=embedding)



```
# This is formatted as code
```

First let's try passing a question through to a single one of these `vecstore` objects:

In [ ]:
from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough
)

retriever_a = chroma_db_a.as_retriever(search_kwargs={"k": min(4, len(docs_a))})
retriever_b = chroma_db_b.as_retriever(search_kwargs={"k": min(4, len(docs_a))})

prompt_str = """Answer the question below using the context:

Context: {context}

Question: {question}

Answer: """
prompt = ChatPromptTemplate.from_template(prompt_str)

retrieval = RunnableParallel(
    {"context": retriever_a, "question": RunnablePassthrough()}
)

chain = retrieval | prompt | llm | output_parser

In [ ]:
out = chain.invoke("when was James born?")
print(out)

James' birthday is on the 7th December.


Here we have used `RunnableParallel` to create two parallel streams of information, one for `"context"` that is information fed in from `retriever_a`, and another for `"question"` which is the _passthrough_ information, ie the information that is passed through from our `chain.invoke("when was James born?")` call.

Using this information the chain is close to answering the question but it doesn't have enough information, it is missing the information that we have stored in `retriever_b`. Fortunately, we can have multiple parallel information streams using the `RunnableParallel` object.

In [ ]:
prompt_str = """Answer the question below using the context:

Context:
{context_a}
{context_b}

Question: {question}

Answer: """
prompt = ChatPromptTemplate.from_template(prompt_str)

retrieval = RunnableParallel(
    {
        "context_a": retriever_a, "context_b": retriever_b,
        "question": RunnablePassthrough()
    }
)

chain = retrieval | prompt | llm | output_parser

In [ ]:
out = chain.invoke("when was James born?")
print(out)

James was born on 7th December 1994.


Now the `RunnableParallel` object is allowing us to search with `retriever_a` _and_ `retriever_b` in parallel, ie at the same time.

## Runnable Lambdas

The `RunnableLambda` is a LangChain abstraction that allows us to turn Python functions into pipe-compatible function, similar to the `Runnable` class we created near the beginning of this notebook.

Let's try it out with our earlier `add_five` and `multiply_by_two` functions.

In [ ]:
from langchain_core.runnables import RunnableLambda

# wrap the functions with RunnableLambda
add_five = RunnableLambda(add_five)
multiply_by_two = RunnableLambda(multiply_by_two)

As with our earlier `Runnable` abstraction, we can use `|` operators to chain together our `RunnableLambda` abstractions.

In [ ]:
chain = add_five | multiply_by_two

Unlike our `Runnable` abstraction, we cannot run the `RunnableLambda` chain by calling it directly, instead we must call `chain.invoke`:

In [ ]:
chain.invoke(3)

16

As before, we can see the same answer. Naturally we can feed custom functions into our chains using this approach. Let's try a short chain and see where we might want to insert a custom function:

In [ ]:
prompt_str = "Tell me an short fact about {topic}"
prompt = ChatPromptTemplate.from_template(prompt_str)

chain = prompt | llm | output_parser

In [ ]:
chain.invoke({"topic": "Artificial Intelligence"})

'Artificial Intelligence (AI) has the capability to learn and adapt over time. Techniques such as machine learning enable AI systems to improve their performance on tasks by analyzing large amounts of data and identifying patterns, without being explicitly programmed for each specific task.'

In [ ]:
chain.invoke({"topic": "Artificial Intelligence"})

'Artificial Intelligence (AI) can learn and make decisions independently. AI systems are trained using large amounts of data, allowing them to recognize patterns and perform tasks such as translating languages, recommending products, and driving autonomous vehicles.'

The returned text always includes the initial `"Here's a short fact about ...\n\n"` — let's add a function to split on double newlines `"\n\n"` and only return the fact itself.

In [ ]:
def extract_fact(x):
    if "\n\n" in x:
        return "\n".join(x.split("\n\n")[1:])
    else:
        return x

get_fact = RunnableLambda(extract_fact)

chain = prompt | llm | output_parser | get_fact

In [ ]:
chain.invoke({"topic": "Artificial Intelligence"})

'Artificial intelligence (AI) can be categorized into two main types: narrow AI, which is designed to perform a narrow task (like facial recognition or internet searches), and general AI, which can understand and reason its environment as a human would. Most existing AI applications, such as voice assistants and recommendation systems, are forms of narrow AI.'

In [ ]:
chain.invoke({"topic": "Artificial Intelligence"})

'Artificial Intelligence (AI) is the branch of computer science that aims to create machines capable of performing tasks that would normally require human intelligence, such as visual perception, speech recognition, decision-making, and language translation. AI can be categorized into two main types: narrow AI, which is designed to perform specific tasks, and general AI, which aims to perform any intellectual task that a human can do.'

Now we're getting well formatted outputs using our final custom `get_fact` function.

---

That covers the essentials you need to getting started and building with the **L**ang**C**hain **E**xpression **L**anguage (LCEL). With it, we can put together chains very easily — and the current focus of the LangChain team is on further LCEL development and support.

The pros and cons of LCEL are varied. Those that love it tend to focus on the minimalist code style, LCEL's support for streaming, parallel operations, and async, and LCEL's nice integration with LangChain's focus on chaining components together.

There are also people that are less fond of LCEL. Typically people will point to it being yet another abstraction on top of an already very abstract library, that the syntax is confusing, against [the Zen of Python](https://peps.python.org/pep-0020/), and requires too much effort to learn new (or uncommon) syntax.

Both viewpoints are entirely valid, LCEL is a very different approach — ofcourse there are major pros and major cons. In either case, if you're willing to spend some time learning the syntax, it allows us to develop very quickly, and with that in mind it's well worth learning.

---